In [1]:
%pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━ 568.9/797.2 MB 26.5 MB/s eta 0:00:09
ERROR: Operation cancelled by user


In [ ]:
%pip install peft transformers protobuf==3.20 bitsandbytes sentencepiece sacrebleu ipython datasets evaluate deepspeed einops  wandb zstandard accelerate jsonlines trl -q

In [ ]:
import torch
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from peft import PeftModel

In [ ]:
# !git clone https://huggingface.co/haoranxu/ALMA-7B /home/jupyter/datasphere/project/models

In [ ]:
model = AutoModelForCausalLM.from_pretrained("haoranxu/X-ALMA-13B-Pretrain", torch_dtype=torch.float16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("haoranxu/X-ALMA-13B-Pretrain", padding_side='left')

In [ ]:
# Add the source sentence into the prompt template
prompt="Translate this from English to Russian:\nEnglish: Hello, how are you!\nRussian:"

# X-ALMA needs chat template but ALMA and ALMA-R don't need it.
# chat_style_prompt = [{"role": "user", "content": prompt}]
# prompt = tokenizer.apply_chat_template(chat_style_prompt, tokenize=False, add_generation_prompt=True)

input_ids = tokenizer(prompt, return_tensors="pt", padding=True, max_length=40, truncation=True).input_ids.cuda()

# Translation
with torch.no_grad():
    generated_ids = model.generate(input_ids=input_ids, num_beams=5, max_new_tokens=20, do_sample=True, temperature=0.8, top_p=0.9)
outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
print(outputs)

In [ ]:
import warnings

warnings.filterwarnings('ignore')

In [ ]:
%pip install pyyaml -q

In [ ]:
import yaml

In [ ]:
# %run ./ALMA/run_cpo_llmmt.py \
#     --config_name /home/jupyter/datasphere/project/ALMA/configs/conf.json \
#     --model_name_or_path haoranxu/X-ALMA-13B-Pretrain \
#     --tokenizer_name haoranxu/X-ALMA-13B-Pretrain \
#     --peft_model_id  haoranxu/ALMA-13B-Pretrain-LoRA \
#     --cpo_scorer kiwi_xcomet \
#     --beta 0.1 \
#     --use_peft \
#     --use_fast_tokenizer False \
#     --cpo_data_path  "marulyanova/ALMA-R-Preference-F" \
#     --do_train \
#     --language_pairs "['ru-en']" \
#     --low_cpu_mem_usage \
#     --bf16 \
#     --learning_rate 1e-4 \
#     --weight_decay 0.01 \
#     --gradient_accumulation_steps 1 \
#     --lr_scheduler_type inverse_sqrt \
#     --warmup_ratio 0.01 \
#     --ignore_pad_token_for_loss \
#     --ignore_prompt_token_for_loss \
#     --per_device_train_batch_size 2 \
#     --evaluation_strategy no \
#     --save_strategy steps \
#     --save_total_limit 1 \
#     --save_steps=150 \
#     --logging_strategy steps \
#     --logging_steps 0.05 \
#     --output_dir dist \
#     --num_train_epochs 1 \
#     --prediction_loss_only \
#     --max_new_tokens 256 \
#     --max_source_length 256 \
#     --max_prompt_length 256 \
#     --max_length 512 \
#     --seed 42 \
#     --overwrite_output_dir \
#     --report_to none \
#     --overwrite_cache

In [ ]:
# --config_name ALMA/configs/conf.json \

In [ ]:
%run ./ALMA/run_cpo_llmmt.py \
    --config_name ALMA/configs/conf.json \
    --model_name_or_path haoranxu/X-ALMA-13B-Pretrain \
    --tokenizer_name haoranxu/X-ALMA-13B-Pretrain \
    --peft_model_id  haoranxu/ALMA-13B-Pretrain-LoRA \
    --cpo_scorer kiwi_xcomet \
    --beta 0.1 \
    --use_peft \
    --use_fast_tokenizer False \
    --cpo_data_path  marulyanova/ALMA-R-Preference-F \
    --do_train \
    --language_pairs "['ru-en']" \
    --low_cpu_mem_usage \
    --fp16 \
    --learning_rate 1e-4 \
    --weight_decay 0.01 \
    --gradient_accumulation_steps 1 \
    --lr_scheduler_type inverse_sqrt \
    --warmup_ratio 0.01 \
    --ignore_pad_token_for_loss \
    --ignore_prompt_token_for_loss \
    --per_device_train_batch_size 1 \
    --evaluation_strategy no \
    --save_strategy steps \
    --save_total_limit 1 \
    --logging_strategy steps \
    --logging_steps 0.05 \
    --output_dir dist \
    --num_train_epochs 1 \
    --prediction_loss_only \
    --max_new_tokens 256 \
    --max_source_length 256 \
    --max_prompt_length 256 \
    --max_length 512 \
    --seed 42 \
    --overwrite_output_dir \
    --report_to none \
    --overwrite_cache \
    --eval_accumulation_steps 1 \
    --per_device_eval_batch_size 1

In [ ]:
%pip install torch transformers --upgrade -q

In [ ]:
# !git clone https://github.com/fe1ixxu/ALMA.git

In [ ]:
import json

# Определяем конфигурацию в виде словаря
config = {
    "compute_environment": "LOCAL_MACHINE",
    "deepspeed_config": {
        "gradient_accumulation_steps": 1,
        "gradient_clipping": 1.0,
        "offload_optimizer_device": "none",
        "offload_param_device": "cpu",
        "zero3_init_flag": False,
        "zero_stage": 2
    },
    "distributed_type": "DEEPSPEED",
    "downcast_bf16": "no",
    "machine_rank": 0,
    "main_training_function": "main",
    "mixed_precision": "bf16",
    "num_machines": 1,
    "num_processes": 8,
    "rdzv_backend": "static",
    "same_network": True,
    "tpu_env": [],
    "tpu_use_cluster": False,
    "tpu_use_sudo": False,
    "use_cpu": False,
    "model_type": "llama",
    "max_position_embeddings": 4096,
    "attention_bias": False,
    "attention_dropout": 0.0,
    "bos_token_id": 1,
    "eos_token_id": 2,
    "head_dim": 128,
    "hidden_act": "silu",
    "hidden_size": 5120,
    "initializer_range": 0.02,
    "intermediate_size": 13824,
    "max_length": 512,
    "max_position_embeddings": 4096,
    "mlp_bias": False,
    "model_type": "llama",
    "num_attention_heads": 40,
    "num_hidden_layers": 40,
    "num_key_value_heads": 40,
    "pad_token_id": 0,
    "pretraining_tp": 1,
    "rms_norm_eps": 1e-05,
    "rope_scaling": None,
    "rope_theta": 10000.0,
    "tie_word_embeddings": False,
    "torch_dtype": "bfloat16",
    "transformers_version": "4.45.2",
    "use_cache": True,
    "vocab_size": 32000
}

# Указываем путь к файлу, в который будем сохранять конфигурацию
json_file_path = '/home/jupyter/datasphere/project/ALMA/configs/conf.json'

# Сохраняем конфигурацию в JSON файл
with open(json_file_path, 'w') as json_file:
    json.dump(config, json_file, indent=4)

print(f"Конфигурация сохранена в файл: {json_file_path}")